In [2]:
import mysql.connector
from collections import Counter
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import jsonlines  # jsonlines 모듈 추가
import re

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# MariaDB 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'port': '3306',
    'database': 'news_final',  # 실제 데이터베이스 이름에 맞게 수정하세요
    'raise_on_warnings': True,
}

# 'summa_news' 테이블 생성 함수
def create_summa_news_table_if_not_exists(cursor):
    cursor.execute("SHOW TABLES LIKE 'summa_news'")
    result = cursor.fetchone()

    if not result:
        cursor.execute("""
            CREATE TABLE summa_news (
                id INT AUTO_INCREMENT PRIMARY KEY,
                news_id INT,
                summary TEXT,
                FOREIGN KEY (news_id) REFERENCES news02(id)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_general_ci
        """)
        print("테이블 'summa_news'가 생성되었습니다.")

# 문장 중요도 계산 함수
def calculate_sentence_importance(content):
    sentences = sent_tokenize(content)
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(content)
    filtered_words = [word.lower() for word in word_tokens if word.isalnum() and word.lower() not in stop_words]
    
    word_frequency = Counter(filtered_words)
    sentence_importance = {sentence: sum(word_frequency[word] for word in word_tokenize(sentence.lower()) if word.isalnum()) for sentence in sentences}
    
    return sentence_importance


# 뉴스 요약 및 데이터베이스에 저장 함수
def summarize_and_save_to_database(news_id, content, num_sentences=2):
    # [기자 이름 기자], (기자 이름 기자), 기자 이름 기자 패턴 찾기 및 제거
    content = re.sub(r'\[.*?\]|\(.*?\)|\s기자\s', '', content)
    
    # '기자' 다음에 나오는 글자들 제거
    content = re.sub(r'기자\s{1,2}[\wㄱ-ㅎㅏ-ㅣ가-힣]{2,5}', '기자', content)
    
    # [기자 이름 기자], (기자 이름 기자), 기자 이름 기자 패턴 찾기 및 제거
    content = re.sub(r'\[.*?(\S+)\s기자\]', r'', content)
    content = re.sub(r'\(.*?(\S+)\s기자\)', r'', content)
    content = re.sub(r'(\S+)\s기자', r'', content)



    # 이메일 주소 제거
    content = re.sub(r'\S+@\S+', '', content)

    # '사진=' 패턴 찾기 및 제거
    content = re.sub(r'사진=', '', content)

    # 특수기호(쉼표와 마침표, 작은따옴표와 큰따옴표 제외) 제거
    content = re.sub(r'[^\w\sㄱ-ㅎㅏ-ㅣ가-힣,.\'"]', '', content)
    content = re.sub(r'\n', '', content)



    try:
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()

        create_summa_news_table_if_not_exists(cursor)

        cursor.execute("SELECT id FROM summa_news WHERE news_id = %s", (news_id,))
        result = cursor.fetchone()

        if not result:
            sentence_importance = calculate_sentence_importance(content)
            sorted_sentences = sorted(sentence_importance, key=sentence_importance.get, reverse=True)

             # 요약할 문장 수가 2줄을 초과하지 않도록 조정
            num_selected_sentences = min(num_sentences, len(sorted_sentences))
            summary = ' '.join(sorted_sentences[:num_selected_sentences])
            
            
            cursor.execute("INSERT INTO summa_news (news_id, summary) VALUES (%s, %s)", (news_id, summary))
            connection.commit()
            print(f"뉴스 ID {news_id}에 대한 요약이 'summa_news' 테이블에 저장되었습니다.")
            
            print(f"뉴스 ID {news_id} 요약 내용: {summary}")

            return summary
        else:
            print(f"뉴스 ID {news_id}에 대한 요약은 이미 존재합니다.")
            return None

    except mysql.connector.Error as err:
        print(f"에러 발생: {err}")
        return None

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MariaDB 연결 종료")

# JSONL 파일에 저장하는 함수 추가
def save_to_jsonl(original_articles, summaries, filename='news_summaries.jsonl'):
    with jsonlines.open(filename, 'w') as writer:
        for original_article, summary in zip(original_articles, summaries):
            data = {'Original Article': original_article, 'Summary': summary}
            writer.write(data)
    print(f"데이터가 '{filename}' 파일에 저장되었습니다.")

# 모든 뉴스에 대해 요약하고 'summa_news' 테이블에 저장
original_articles = []
summaries = []

try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()

    cursor.execute("SELECT id, content FROM news02")
    news_articles = cursor.fetchall()

    for article_id, article_content in news_articles:
        summary = summarize_and_save_to_database(article_id, article_content)
        
        if summary is not None:
            original_articles.append(article_content)
            summaries.append(summary)

except mysql.connector.Error as err:
    print(f"에러 발생: {err}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MariaDB 연결 종료")

# JSONL 파일에 저장
save_to_jsonl(original_articles, summaries)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


테이블 'summa_news01'가 생성되었습니다.
뉴스 ID 1에 대한 요약이 'summa_news' 테이블에 저장되었습니다.
뉴스 ID 1 요약 내용: "명단에 들어가 있지 않는 선배 정치인 분들은 후배들을 위해 길을 터줄 수 있도록 책임있는 결정을"국민의힘은 부산경남 중진인 5선 서병수 의원과 3선 김태호 의원에게 민주당 현역 의원 지역구로 출마할 것을 요청했습니다. 앵커 4월 총선을 두 달여 앞두고 여야의 지역구 공천이 윤곽을 드러내고 있습니다.민주당은 경선 지역구 23곳과 단수 공천지역 13곳을 결정했고, 국민의힘은 부산 경남 중진 의원들에게 험지 출마를 입니다.
MariaDB 연결 종료
뉴스 ID 2에 대한 요약이 'summa_news' 테이블에 저장되었습니다.
뉴스 ID 2 요약 내용: 월가의 대형 투자은행들이 앞으로 10년 동안 인도를 주요 투자 대상으로 선택하면서 현금의 대부분이 중국이 아닌 인도로 향하는 변화가 일어나고 있다.블룸버그는 5일 보도에서 아시아에서 인도를 향한 골드러시가 일어나고 있다고 전했다.이어 취리히에 본사를 둔 본토벨 홀딩스도 인도를 신흥시장 1위 기업으로 선정했으며, 전통적으로 보수적인 모습을 모이던 일본 개인 투자자들도 중국의 노출을 줄이고 인도를 받아들이고 있다.월가의 대형 투자은행들이 앞으로 10년 동안 인도를 주요 투자 대상으로 선택하면서 현금의 대부분이 중국이 아닌 인도로 향하는 변화가 일어나고 있다.블룸버그는 5일 보도에서 아시아에서 인도를 향한 골드러시가 일어나고 있다고 전했다.620억 달러 규모의 헤지펀드 마셜 웨스가 미국에 이어 인도를 최대 순매수 자산으로 꼽았다. 이어 취리히에 본사를 둔 본토벨 홀딩스도 인도를 신흥시장 1위 기업으로 선정했으며, 전통적으로 보수적인 모습을 모이던 일본 개인 투자자들도 중국의 노출을 줄이고 인도를 받아들이고 있다.세계에서 가장 빠르게 성장하는 주요 경제국인 인도는 베이징으로부터 세계 자본과 공급망을 끌어들이기 위해 나렌드라 모디 총리의 통치하에 인프라를 대폭 확장했다.
M